# Notebook de Estrutura de Dados e Algoritmos de Ordenação

Centro de Ensino Unificado de Brasília - UniCeub

Professor: Pedro Henrique Rodrigues Mendes

2° Semestre de 2022

**Curso:** Ciência de Dados e Machinelearning - 2º semestre

**Aluno:** Guilherme Dias Barbosa - 22209454

## Introdução

Área onde o aluno deverá detalhar e introdudizr o leitor ao problema: [Compatilhamento de Bicicletas em Brasília](https://www.kaggle.com/datasets/joseguilhermelopes/bike-sharing-system-in-brasilia-brazil). 

A abordagem dever ser feita nos seguintes pontos:
- **História do serviço de bicicletas compartilhadas em Brasilia;**
- **Gloassário de dados (Descrever os campos dos datasets).**


## Importação dos Dados

In [91]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [92]:
df_rides = pd.read_csv('df_rides.csv', sep=',')
df_stations = pd.read_csv('df_stations.csv', sep=',')

## Formatação dos Dados

In [93]:
# transformando a coluna user_birthdate no tipo data
df_rides['user_birthdate'] = df_rides['user_birthdate'].astype("datetime64")

## Listas e Filas

O Aluno deverá implementar um sistema de **Fila (first-in first-out) e Pilha (last in, first out)** com base na classe "rides" abaixo. 

O objetivo é criar um "buffer de requisições" utilizando a Classe Ride, portanto, o usuário quando solicita um serviço de bicicleta, a requisição deve organizada em uma Fila (first-in first-out) ou Pilha (last in, first out) para ser processada no sistema.

**A classe Ride pode ser alterada para adequação ao sistemas de fila e pilha, sempre detalhando as mudanças no código.**

**Sempre comentando código e detalhando o embasamento teórico para implementação da técnica.** 

In [102]:
if '1 - Memorial JK' in df_stations.station.unique():
            print("ok")
else:
            print("Não Possui essa informação de estação na base da dados")
            status = False    

ok


In [119]:

class ride:
    
    uf_array = ['AP', 'DF', 'SP', 'RS', 'PR', 'RJ', 'GO', 'AC',
                'MA', 'PA', 'MG', 'RN', 'PE', 'MT', 'ES', 'PB', 
                'RR', 'BA', 'AL', 'PI', 'CE', 'SC','RO', 'AM', 
                'MS', 'TO', 'SE'] 
    
    gender_array = ['M','F']
    
    def __init__(self, gender, birthdate, residence, station_start):
        
        self.db = df_rides
        self.db_stations = df_stations
        
        status = True
        
        if gender in ride.gender_array:
            pass
        else:
            status = False
            print("Não Possui essa informação de genero na base da dados")
            
        if  station_start in self.db_stations.station.unique():
            pass
        else:
            print("Não Possui essa informação de estação na base da dados")
            status = False    
            
        if  residence in ride.uf_array:
            pass
        else:
            print("Não Possui essa informação de UF brasileiro na base da dados")
            status = False 
        
        if status:
            self.status = "Iniciado"
            self.gender = gender
            self.birthdate = birthdate
            self.residence = residence
            self.ride_date =  pd.Timestamp.now().strftime('%Y-%m-%d')
            self.time_start = pd.Timestamp.now().strftime('%H:%M:%S')
            self.station_start = station_start
            
            
            self.index = len(self.db.index)
            
            self.time_end = None
            self.station_end = None
            self.ride_duration = None
            self.ride_late = None
            
            insert_list = [self.gender,
                          self.birthdate,
                          self.residence,
                          self.ride_date,
                          self.time_start,
                          self.time_end,
                          self.station_start,
                          self.station_end,
                          self.ride_duration,
                          self.ride_late]
            
            print(insert_list)
            self.db.loc[self.index] = insert_list
            
        else:
            print('Erro de Validação ou insert na base')
                
    
    def user_age(self):
        print(int(str((pd.Timestamp.now() - pd.to_datetime(self.birthdate, format='%Y-%m-%d') ) /365)[:2].strip()))
        
    def get(self):
        
        get_list = [self.gender,
                    self.birthdate,
                    self.residence,
                    self.ride_date,
                    self.time_start,
                    self.time_end,
                    self.station_start,
                    self.station_end,
                    self.ride_duration,
                    self.ride_late]
        
        print(get_list)
        
    def finish_ride(self, station_end):
        
        status =  True
        
        if  station_end in self.db_stations.station.unique():
            pass
        else:
            print("Não Possui essa informação como padrão na base da dados")
            status = False
        
        if status:
            
            self.status = "Concluido"
            self.time_end = pd.Timestamp.now().strftime('%H:%M:%S')
            self.station_end =  station_end
            self.ride_duration = pd.Timedelta(pd.Timestamp.now() - pd.to_datetime(self.ride_date + ' ' + self.time_end)) / pd.Timedelta('60s')
            
            if self.ride_duration >= 60:
                self.ride_late =   1
            else:
                self.ride_late =   0
            
            insert_list = [self.gender,
                          self.birthdate,
                          self.residence,
                          self.ride_date,
                          self.time_start,
                          self.time_end,
                          self.station_start,
                          self.station_end,
                          self.ride_duration,
                          self.ride_late]
            
            print(insert_list)
            self.db.loc[self.index] = insert_list
            
        else:
            print('Erro de Validação ou insert na base')
    
    #Aqui é criado uma função para a inserção dos objetos na lista      
    # def insere_na_lista(self):
    #     if 'lista_requisicoes' in globals():
    #         globals()['lista_requisicoes'].append(self) 
    #     else:
    #         globals()['lista_requisicoes'] = [] 
    #         globals()['lista_requisicoes'].append(self) 
             
            

### Fila (first-in first-out)

In [137]:
import random
from pprint import pprint
from re import L

indice_criacao = 0

#Essa é 
def ordem():  
    lista_req = []
    for i in range(10):
        random_ride = (random.choice(ride.gender_array),
                       str(df_rides.user_birthdate[random.randint(0,287321)])[0:10],
                       random.choice(ride.uf_array),
                       str(df_rides.station_start[random.randint(0,287320)]).strip()
        ) 
        lista_req.append(random_ride)    
    return lista_req

requisicoes = ordem()
args_req = [None] * 4
print("Lista de requisições:", len(requisicoes),"\n")

for i in range(len(requisicoes)):
    for x in range(4):
        args_req[x] = str(requisicoes[0][x])
    ride(args_req[0],
         args_req[1],
         args_req[2],
         args_req[3])
    requisicoes.pop(0)
       
print("\nLista de requisições:", len(requisicoes))

Lista de requisições: 10 

['F', '1996-11-18', 'RO', '2022-09-16', '15:19:43', None, '41 - Instituto de Artes', None, None, None]
['M', '2001-01-16', 'SC', '2022-09-16', '15:19:43', None, '42 - PAT', None, None, None]
['M', '1996-05-09', 'AM', '2022-09-16', '15:19:43', None, '1 - Memorial JK', None, None, None]
['F', '1992-02-12', 'PE', '2022-09-16', '15:19:43', None, '21 - EQS 202 / 203', None, None, None]
['M', '1991-01-02', 'GO', '2022-09-16', '15:19:43', None, '1 - Memorial JK', None, None, None]
['M', '1988-10-29', 'TO', '2022-09-16', '15:19:43', None, '24 - Galeria', None, None, None]
['M', '1998-06-22', 'MS', '2022-09-16', '15:19:43', None, '1 - Memorial JK', None, None, None]
['F', '1992-02-08', 'SP', '2022-09-16', '15:19:43', None, '45 - Centro Olímpico', None, None, None]
['M', '1996-02-06', 'BA', '2022-09-16', '15:19:43', None, '27 - Ministério do Planejamento', None, None, None]
['F', '1982-02-20', 'RR', '2022-09-16', '15:19:43', None, '2 - Praça Buriti', None, None, None]


### Pilha (last in, first out)

## Algortimos de Ordenação
O Aluno desenvolve os algoritmos de ordenação abaixo, com o objetivo de ordenar o dataset **df_stations** embaralhado.

**Realizar a Ánalise de Complexidade nos algoritmos (Notação Big O e Melhor Caso/Pior Caso)**

**Sempre comentando código e detalhando o embasamento teórico para implementação da técnica.** 

In [ ]:
df_stations.head(5)

In [ ]:
df_stations = df_stations.sample(frac=1,random_state=42)

In [ ]:
df_stations.head(5)

### Bubblesort

### Mergesort

### Heapsort

### Quicksort